# Visualized Result from Predicted Articulation
- created: 12 Apr 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import librosa.display
from matplotlib import cm
from functools import partial
import seaborn as sns

import sklearn
from os import makedirs
import os, sys
from os.path import join
import shutil
from IPython import display
import scipy.stats
import fn_tools as utils
%matplotlib inline

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

## Compare Model 
- Compare model on predefined dataset

In [ ]:
# monosyllable
df_mono_result, df_mono_raw = utils.compute_formant_result_table([0,1,38, 37, 3, 71])
df_mono_result

In [ ]:
plt.title('MONOSYLLABIC F1 ERROR')
sns.boxplot(y='F1 ERROR', x='MODEL', data=df_mono_raw[df_mono_raw['MODEL'] != 0.0], width=0.8,palette="muted")
plt.savefig('monosyllabic_f1_error.png')

In [ ]:
plt.title('MONOSYLLABIC F2 ERROR')
sns.boxplot(y='F2 ERROR', x='MODEL', data=df_mono_raw[df_mono_raw['MODEL'] != 0.0], width=0.8, palette="muted")
plt.savefig('monosyllabic_f2_error.png')

In [ ]:
# disyllable
df_di_result, df_di_raw = utils.compute_formant_result_table([7,8,30,31,10,27 ])
# df_di_result, df_di_raw = utils.compute_formant_result_table([57, 58,63,59,60,61,62 ])
df_di_result

In [ ]:
plt.title('DISYLLABIC F1 ERROR')
sns.boxplot(y='F1 ERROR', x='MODEL', data=df_di_raw[df_di_raw['MODEL'] != 7.0], width=0.8,palette="muted")
plt.savefig('disyllabic_f1_error.png')

In [ ]:
plt.title('DISYLLABIC F2 ERROR')
sns.boxplot(y='F2 ERROR', x='MODEL', data=df_di_raw[df_di_raw['MODEL'] != 7.0], width=0.8,palette="muted")
plt.savefig('disyllabic_f2_error.png')

## Eval Mono Vowel

In [ ]:
exp_num = 71
utils.compute_each_vowel_formant_eval_mono(exp_num)

## Eval Disyllablic vowel utterance 

In [ ]:
exp_num = 27
res_df = utils.compute_each_vowel_formant_eval_di(exp_num)
res_df.head()

In [ ]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

utils.change_label_set_1(res_1, col='Pho1')
res_1 = utils.set_datapoint_index(res_1, col='Pho1')
utils.custome_reindex_type2(res_1).drop(['Pho1'], axis=1)

In [ ]:
res_2 = res_df.groupby(['Pho2']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

utils.change_label_set_1(res_2, col='Pho2')
res_2 = utils.set_datapoint_index(res_2, col='Pho2')
utils.custome_reindex_type2(res_2).drop(['Pho2'], axis=1)

## Predict Disyllablic vowel utterance 

In [66]:
exp_num = '27_3'
res_df = utils.compute_each_vowel_formant_predict_di(exp_num)
res_df.head()

,Pho,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR,Pho1,Pho2
0,a;i,15.505360,11.191572,24.553149,23.551074,11.232114,13.918637,a,i
1,a;u,15.684997,18.692738,21.025527,14.367761,10.274503,3.922319,a,u
2,a;e,40.568789,10.458509,29.908192,25.679888,10.402088,2.688048,a,e
3,a;E,17.339106,9.027184,21.114030,11.663415,20.671985,7.555845,a,E
4,a;M,12.817191,12.460444,24.888904,10.933250,4.032835,14.691795,a,M


In [67]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

utils.change_label_set_2(res_1, col='Pho1')
res_1 = utils.set_datapoint_index(res_1, col='Pho1')
utils.custome_reindex_type3(res_1).drop(['Pho1'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,20.534994,15.139364,15.537696
i:,20.993131,31.225375,13.550929
u:,17.345918,29.233214,11.077172
e:,15.119441,18.570880,13.621373
ɛ:,16.010844,26.188627,14.268820
ɯ:,12.131145,13.445872,14.419638
ɤ:,14.045857,11.363696,13.860680
o:,20.808649,23.735418,11.826913
ɔ:,14.295219,17.227784,13.800858


In [68]:
res_2 = res_df.groupby(['Pho2']).mean()[['2F1 ERROR', '2F2 ERROR','2F3 ERROR']]

utils.change_label_set_2(res_2, col='Pho2')
res_2 = utils.set_datapoint_index(res_2, col='Pho2')
utils.custome_reindex_type3(res_2).drop(['Pho2'], axis=1)

,2F1 ERROR,2F2 ERROR,2F3 ERROR
Label_idx,,,
a:,23.454735,14.539714,16.192348
i:,17.809831,25.590296,14.524220
u:,46.025272,59.956700,19.515814
e:,19.208995,18.591556,14.250078
ɛ:,21.894402,22.875184,15.169592
ɯ:,15.089493,14.028627,14.483900
ɤ:,21.350277,11.408602,13.431068
o:,43.862274,50.971898,17.349717
ɔ:,19.427060,17.417680,13.004481


## Predict Formant Range

In [59]:
def get_min_max_mean_each_vowel_formant_predict_di(exp_num):
    path = '../../experiment/result/predict_{}/formant/'.format(exp_num)


    with open('../../data/d_records/record_all/syllable_name.txt') as f:
        label = np.array([word.strip()[0]+';'+word.strip()[1] for line in f for word in line.split(',')])

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))

    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))

    a1 = utils.cut_head_and_tail(utils.numpy_fillna(a1)).astype('float')
    a2 = utils.cut_head_and_tail(utils.numpy_fillna(a2)).astype('float')

    e1 = utils.cut_head_and_tail(utils.numpy_fillna(e1)).astype('float')
    e2 = utils.cut_head_and_tail(utils.numpy_fillna(e2)).astype('float')
    
    e1 = e1.reshape((e1.shape[0]//2,2,e1.shape[1]))
    e2 = e2.reshape((e2.shape[0]//2,2,e2.shape[1]))
    
    a11_min = np.min(e1[:,0,:], axis=1)
    a12_min = np.min(e2[:,0,:], axis=1)
    a21_min = np.min(e1[:,1,:], axis=1)
    a22_min = np.min(e2[:,1,:], axis=1)

    a11_max = np.max(e1[:,0,:], axis=1)
    a12_max = np.max(e2[:,0,:], axis=1)
    a21_max = np.max(e1[:,1,:], axis=1)
    a22_max = np.max(e2[:,1,:], axis=1)
    
    a11_mean = np.median(e1[:,0,:], axis=1)
    a12_mean = np.median(e2[:,0,:], axis=1)
    a21_mean = np.median(e1[:,1,:], axis=1)
    a22_mean = np.median(e2[:,1,:], axis=1)
 


    res_df = pd.DataFrame({ 'Pho':label, 
                           '1F1 MIN': a11_min,'1F2 MIN':a12_min, '2F1 MIN': a21_min,'2F2 MIN':a22_min,
                           '1F1 MAX': a11_max,'1F2 MAX':a12_max, '2F1 MAX': a21_max,'2F2 MAX':a22_max,
                           '1F1 MEAN': a11_mean,'1F2 MEAN':a12_mean, '2F1 MEAN': a21_mean,'2F2 MEAN':a22_mean
                          })

    res_df['Pho1'], res_df['Pho2'] = res_df['Pho'].str.split(';', 1).str
    
    return res_df

In [63]:
exp_num = '27_3'
res_df = get_min_max_mean_each_vowel_formant_predict_di(exp_num)
res_df.head()

,Pho,1F1 MIN,1F2 MIN,2F1 MIN,2F2 MIN,1F1 MAX,1F2 MAX,2F1 MAX,2F2 MAX,1F1 MEAN,1F2 MEAN,2F1 MEAN,2F2 MEAN,Pho1,Pho2
0,a;i,649.639601,1389.246039,273.744246,2179.194967,652.320412,1395.656933,277.530392,2190.370863,650.802755,1392.959430,275.819489,2186.525878,a,i
1,a;u,617.526375,1246.784854,348.276166,683.484781,623.188156,1255.160006,382.110867,830.510723,619.307688,1251.288258,352.343322,688.669992,a,u
2,a;e,466.092788,1672.233702,371.601454,2001.498373,471.194767,1713.378005,379.464963,2036.332547,467.955921,1682.302231,372.163167,2032.075812,a,e
3,a;E,676.224015,1373.378749,609.254850,1654.098547,681.572364,1375.931790,611.402256,1662.868669,677.749665,1374.969353,609.967210,1661.398069,a,E
4,a;M,650.945367,1320.515855,341.214877,1556.269629,656.094590,1332.439816,344.396142,1582.599041,652.137846,1323.729516,341.752747,1573.037155,a,M


In [64]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 MEAN', '1F2 MEAN']]

utils.change_label_set_2(res_1, col='Pho1')
res_1 = utils.set_datapoint_index(res_1, col='Pho1')
utils.custome_reindex_type3(res_1).drop(['Pho1'], axis=1)

,1F1 MEAN,1F2 MEAN
Label_idx,,
a:,574.525209,1220.572956
i:,271.320508,1696.538765
u:,332.131732,770.168233
e:,383.219567,1650.299519
ɛ:,535.784626,1464.752360
ɯ:,336.791393,1310.571450
ɤ:,471.243177,1313.409550
o:,423.990145,791.917948
ɔ:,577.741123,1081.271072


In [65]:
res_1 = res_df.groupby(['Pho2']).mean()[['2F1 MEAN', '2F2 MEAN']]

utils.change_label_set_2(res_1, col='Pho1')
res_1 = utils.set_datapoint_index(res_1, col='Pho1')
utils.custome_reindex_type3(res_1).drop(['Pho1'], axis=1)

,2F1 MEAN,2F2 MEAN
Label_idx,,
a:,594.933324,1251.761268
i:,268.101852,1928.079941
u:,465.829827,1088.226722
e:,371.314433,1726.161851
ɛ:,552.740457,1446.581639
ɯ:,300.964135,1304.823136
ɤ:,441.059017,1266.954556
o:,489.313857,980.069271
ɔ:,578.219185,1041.712819
